In [ ]:
# 고정된 데이터로만 했으니까 이제는 데이터를 n개로 분할하여, 분할된 데이터를 매 검증 때마다 번갈아가며 사용
# 모델이 얼마나 잘 일반화 되어있는지 확인할 때 교차검증 사용
# 1. 클래스 이용 2. 함수(매서드) 이용

In [ ]:
# 범주형 데이터를 이용하여 분류 분석용 모델을 만드는 경우에는
# StratifiedKFold가 더 정확하다

# 패키지 로딩

In [2]:
from sklearn.datasets import load_iris
from sklearn.model_selection import KFold, StratifiedKFold # 교차검증
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate # cross_val_score, cross_validate : 교차검증 함수
from sklearn.tree import DecisionTreeClassifier # 분류 문제를 해결하기 위한 모델 객체 (Linear Regression과 비슷한 것)
from sklearn.metrics import accuracy_score # 분류 모델 성능 측정용 : 분류 문제 관련한 모형들에 대한 검증 방법 중 하나가 얼마나 분류를 잘 했는지를 보는 것임

import numpy as np
import pandas as pd

# 데이터 로딩

In [3]:
iris = load_iris()
print(iris.data.shape)
print(iris.target)

(150, 4)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


# K-Fold Cross Validation

In [7]:
k_fold = KFold(n_splits=5)
x = iris.data
y = iris.target
k = 0 # 교차검증 횟수
cv_acc = [] # cross validation accuracy : 교차 검증마다의 정확도를 저장하는 리스트

model = DecisionTreeClassifier(random_state=1) # 모델 생성
# random_state=1 : 의사결정나무(DecisionTreeClassifier)를 생성할 때, 나무에 사용할 샘플 데이터를 추출한다. 이때 랜덤 추출되는 샘플 데이터의 seed 값을 고정한 것

for train_index, test_index in k_fold.split(x):
    # k_fold.split(x) : 총 150개의 데이터를 (n_splits=5이므로) 5개로 나누고 순서대로 abcde그룹이라고 가정해보자
    # k=1일 때, abcd가 train + e가 test가 되며, 이때의 index를 가져온다
    # k=2일 때, bcde가 train + a가 test가 되며, 이때의 index를 가져온다
    # k=3일 때, cdea가 train + b가 test가 되며, 이때의 index를 가져온다
    # k=4일 때, deab가 train + c가 test가 되며, 이때의 index를 가져온다
    # k=5일 때, eabc가 train + d가 test가 되며, 이때의 index를 가져온다
    # 따라서 모든 5개의 그룹이 한 번씩 다 검증에 사용된다.
    
    # 5개로 등분 후 학습/평가 그룹으로 나누어진 데이터들의 인덱스 값을 반환하기
    k += 1
    print(f'{k}번째 검증 데이터 세트')
    print('Train: ', train_index)
    print('Test: ', test_index)
    # 학습 데이터와 평가 데이터를 분할하여 가져오기
    x_train, x_test = x[train_index], x[test_index] # 실제 데이터에 인덱싱 
    y_train, y_test = y[train_index], y[test_index] # 실제 데이터에 인덱싱
    
    model.fit(x_train, y_train) # 모델 학습
    y_hat = model.predict(x_test) # 모델 예측
    acc = accuracy_score(y_test, y_hat) # 모델 성능 측정 및 평가 : 정확도
    print(f'{k}번째 교차 검증 정확도: {acc:.2f}')
    cv_acc.append(acc) # 매 교차검증 마다의 평균 정확도를 확인하기 위해 만든 것
    print('-'*30)

print(f'평균 정확도: {np.mean(cv_acc):.2f}')

# 1번째 검증 데이터 세트
# Train : 첫 번째 검증에 사용된 학습 데이터들의 인덱스 값 : 총 120개
# Test : 첫 번째 검증에 사용된 평가 데이터들의 인덱스 값 : 총 30개
# 1번째 교차 검증 정확도: 1.00 : 다 맞춘 것 : 품종 100% 다 맞춘 것!
# 평균 정확도: 0.91

# 이전에 배웠던 대로 고정된 dataset으로만 하면 분류형일 경우에는 특정 그룹 데이터만, 즉 편향된 데이터만 사용하게 된다. 
# 계층화된 k-flod class validation을 사용하면, 데이터 셋의 모든 것을 활용하여 만들 수 있다 -> 얼마나 일반화가 잘 되어있는지 체크해주는 것이 평균 정확도: 0.91

# 회귀분석에서 k-flod class validation을 사용할 때는 전혀 문제가 되지 않는다.

1번째 검증 데이터 세트
Train:  [ 30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47
  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65
  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83
  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119
 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]
Test:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]
1번째 교차 검증 정확도: 1.00
------------------------------
2번째 검증 데이터 세트
Train:  [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  60  61  62  63  64  65
  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83
  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101
 102 103 104 105 106 107 

# Stratified K-Fold Cross Validation

In [9]:
k_fold = StratifiedKFold(n_splits=5)
x = iris.data
y = iris.target
k = 0 # 교차검증 횟수
cv_acc = [] # cross validation accuracy : 교차 검증마다의 정확도를 저장하는 리스트

model = DecisionTreeClassifier(random_state=1) # 모델 생성
# random_state=1 : 의사결정나무(DecisionTreeClassifier)를 생성할 때, 나무에 사용할 샘플 데이터를 추출한다. 이때 랜덤 추출되는 샘플 데이터의 seed 값을 고정한 것

for train_index, test_index in k_fold.split(x, y):
    # 매개변수에 독립변수 뿐만 아니라 종속변수도 넣어줘야, 모든 데이터를 똑같은 비율로 나눌 수 있다.
    # k_fold.split(x, y) : 총 150개의 데이터는 3개 품종이므로 ABC의 그룹이라고 가정하고, 그것을 (n_splits=5이므로) 5개로 나누어 순서대로 abcde그룹이라고 가정해보자 (ABC는 각 50개 / abcde 그룹은 각 10개)
    # k=1일 때, A그룹의 a, B그룹의 a, C그룹의 a가 test + 나머지는 train
    # k=2일 때, A그룹의 b, B그룹의 b, C그룹의 b가 test + 나머지는 train
    # k=3일 때, A그룹의 c, B그룹의 c, C그룹의 c가 test + 나머지는 train
    # k=4일 때, A그룹의 d, B그룹의 d, C그룹의 d가 test + 나머지는 train
    # k=5일 때, A그룹의 e, B그룹의 e, C그룹의 e가 test + 나머지는 train
    
    # 각 품종별로 5개로 등분 후 학습/평가 그룹으로 나누어진 데이터들의 인덱스 값을 반환하기
    k += 1
    print(f'{k}번째 검증 데이터 세트')
    print('Train: ', train_index)
    print('Test: ', test_index)
    # 학습 데이터와 평가 데이터를 분할하여 가져오기
    x_train, x_test = x[train_index], x[test_index] # 실제 데이터에 인덱싱 
    y_train, y_test = y[train_index], y[test_index] # 실제 데이터에 인덱싱
    
    model.fit(x_train, y_train) # 모델 학습
    y_hat = model.predict(x_test) # 모델 예측
    acc = accuracy_score(y_test, y_hat) # 모델 성능 측정 및 평가 : 정확도
    print(f'{k}번째 교차 검증 정확도: {acc:.2f}')
    cv_acc.append(acc) # 매 교차검증 마다의 평균 정확도를 확인하기 위해 만든 것
    print('-'*30)

print(f'평균 정확도: {np.mean(cv_acc):.2f}')

# stratified k-fold의 평균 정확도가 더 높다!!

1번째 검증 데이터 세트
Train:  [ 10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27
  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45
  46  47  48  49  60  61  62  63  64  65  66  67  68  69  70  71  72  73
  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91
  92  93  94  95  96  97  98  99 110 111 112 113 114 115 116 117 118 119
 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]
Test:  [  0   1   2   3   4   5   6   7   8   9  50  51  52  53  54  55  56  57
  58  59 100 101 102 103 104 105 106 107 108 109]
1번째 교차 검증 정확도: 0.97
------------------------------
2번째 검증 데이터 세트
Train:  [  0   1   2   3   4   5   6   7   8   9  20  21  22  23  24  25  26  27
  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45
  46  47  48  49  50  51  52  53  54  55  56  57  58  59  70  71  72  73
  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90

# cross_val_score()
- estimator : 학습에 사용할 모델 객체를 넣어줌
    - 회귀 문제를 해결하기 위한 모델 객체면 알아서 KFold로 교차검증함
    - 분류 문제를 해결하기 위한 모델 객체면 알아서 StratifiedKFold로 교차검증함
- x : 독립변수 데이터 셋
- y : 종속변수 데이터 셋
- cv : 교차검증 k 값 (default:5) : 분할하고자하는 fold의 개수

In [11]:
score = cross_val_score(model, x, y)
# cv값 없으므로 DEFAULT인 k=5가 적용됨
# model = DecisionTreeClassifier(random_state=1)
# DecisionTreeClassifier : 분류 문제를 해결하기 위한 모델 객체 (Linear Regression은 회귀 문제 해결용)
# random_state=1 : 의사결정나무(DecisionTreeClassifier)를 생성할 때, 나무에 사용할 샘플 데이터를 추출한다. 이때 랜덤 추출되는 샘플 데이터의 seed 값을 고정한 것
print('score: ', score)
print(f'평균 정확도: {np.mean(score):.2f}')

score:  [0.96666667 0.96666667 0.9        1.         1.        ]
평균 정확도: 0.97


# cross_validate()
- cross_val_score()와 유사
- 차이점1 : 매 검증마다 소요된 학습 시간과 검증 시간을 딕셔너리로 반환한다.
- 차이점2 : 여러가지 평가지표를 리스트 형태로 전달 가능 (회귀모델의 평가지표에 MAE, RMSE, R스퀘어가 있듯이, 분류모델의 평가지표에는 정확도, 정밀도, AUC, F1 등이 있다)


In [12]:
result = cross_validate(model, x, y, cv=5, scoring=['accuracy', 'precision_macro', 'roc_auc_ovr'])
# scoring : 평가지표의 이름을 리스트로 전달
# accuracy 정확도 / precision_macro 정밀도 / roc_auc_ovr

for key, val in result.items():
    # 딕셔너리 객체로 반환된 result를 items 함수를 통해 key와 value를 모두 받기
    print('평가지표: ', key)
    print(f'평균값: {np.mean(val):.2f}')
    print('-'*30)

# fit_time : 학습시간
# score_time : 검증시간
# 나머지 : 1에 가까울 수록 좋다.

평가지표:  fit_time
평균값: 0.00
------------------------------
평가지표:  score_time
평균값: 0.00
------------------------------
평가지표:  test_accuracy
평균값: 0.97
------------------------------
평가지표:  test_precision_macro
평균값: 0.97
------------------------------
평가지표:  test_roc_auc_ovr
평균값: 0.97
------------------------------
